In [2]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from numba import njit
from scipy.optimize import curve_fit
import scipy.stats

In [3]:
@njit()
def compute_signature_plot(vol_size,array):
    sigma = np.zeros(vol_size)
    for i in range(1,vol_size+1):
        new_arr = array[::i]
        square_sum = (new_arr[1:] - new_arr[:-1])**2
        sigma[i] = new_arr.mean() / i
        
    return np.sqrt(sigma)

@njit()
def compute_price_impact(price,sign,max_lag,step=1):
    impact = []
    std_impact = []
    for i in range(1,max_lag,step):
        impact.append((-sign[:-i]*(price[i:]-price[:-i])).mean())
        std_impact.append((-sign[:-i]*(price[i:]-price[:-i])).std() / np.sqrt(len(price[i:])))
    
    return impact, std_impact
    
    plt.style.use('dark_background')

In [ ]:
#create a list of all the files in the folder
DIR ="C:\\Users\\spina\\Documents\\SOLDI\\data\\tesla_2015\\order\\"
tesla_files =os.listdir(DIR)
header_list = ["ask price","ask vol","bid price","bid vol"]

#create a list of all the files in the folder
DIR_1 ="C:\\Users\\spina\\Documents\\SOLDI\\data\\tesla_2015\\message\\"
tesla_files_1 =os.listdir(DIR_1)

header_list_1 = ["time","event type","order ID","size","price","direction"]
lst_message = []
lst_order = []
# crate a new dataframe that shows the state of the LOB each second
for message, order in zip(tesla_files_1, tesla_files):
    #load message dataframe
    df_m = pd.read_csv(DIR_1 + message, names = header_list_1)
    # delete first hour and last 30 minutes of trading
    df_m = df_message[df_message["time"]> 37800]
    df_m = df_message[df_message["time"]< 55800]
    #consider the dataframe of all the market order
    df_market = df_m[df_m["event type"] == 4]
    df_market["time"] = (df_market["time"]*100) // 1
    value = df_market["time"].iat[0]
    unique_orders = []
    #iterate over all the row in the dataframe and check if there is a difference higher
    # than 1/100 sec between two market order, if this is not the case the two orders
    # are considered the same
    for i,element in enumerate(df_market["time"]):
        if element == value:
            j = i + 1
            while value == element and j < len(df_market):
                value = df_market["time"].iat[j]
                j += 1
            unique_orders.append(i)
    
    df_market = df_market.iloc[unique_orders]
    market_orders = df_market.index.to_numpy() - 1
    # load order dataframe
    df_o = pd.read_csv(DIR + order, names=header_list,usecols=[0,1,2,3])
    df_o = df_o.iloc[secs] 
    #scale price to dollar cent
    df_o["ask price"] = df_o["ask price"]/100
    df_o["bid price"] = df_o["bid price"]/100
    # add spread and mid price
    df_o["spread"] = df_o["ask price"] - df_o["bid price"]
    df_o["mid price"] = (df_o["ask price"] + df_o["bid price"]) / 2
    #
    df_o = df_o.iloc[market_orders]
    
    lst_message.append(df_market)
    lst_order.append(df_o)